# Лаба 1. Расчет рейтингов фильмов – RDD

## Дедлайн

Понедельник, 1 марта, 23:59:59

## Задача

По имеющимся данным о рейтингах фильмов (MovieLens: 100 000 рейтингов) посчитать агрегированную статистику по ним.

## Описание данных

Имеются следующие входные данные:

* Таблица `users x movies` с рейтингами. Архив с датасетом нужно скачать с сайта [GroupLens](http://files.grouplens.org/datasets/movielens/ml-100k.zip). Также, он загружен на HDFS в `/labs/laba01/ml-100k`. Файл u.data содержит все оценки, а файл u.item — список всех фильмов.

`!hdfs dfs -ls /labs/laba01/ml-100k`

* `id фильма` для расчета индивидуальных характеристик — в Личном кабинете на странице [Лабы 1](https://lk-spark.newprolab.com/lab/slaba01).

## Результат

Выходной формат файла — json. Пример решения:

```json
{
   "hist_film": [  
      134,
      123,
      782,
      356,
      148
   ],
   "hist_all": [  
      134,
      123,
      782,
      356,
      148
   ]
}
```

В поле `“hist_film”` нужно указать для заданного `id` фильма количество поставленных оценок в следующем порядке: `"1", "2", "3", "4", "5"`. То есть сколько было единичек, двоек, троек и т.д.

В поле `“hist_all”` нужно указать то же самое только для всех фильмов общее количество поставленных оценок в том же порядке: `"1", "2", "3", "4", "5"`.

## Проверка

Файл необходимо положить в свою домашнюю директорию на кластере под названием: `lab01.json`.

Проверка осуществляется автоматическим скриптом на странице лабы в личном кабинете.

Обязательное условие зачета лабораторной работы – это выкладка после дедлайна лабы своего решения в репозиторий через pull-request. Как это сделать, можно прочитать [здесь](/git.md). Если будут вопросы – спрашивайте в Slack.


In [2]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [3]:
from pyspark import SparkContext, SparkConf

conf = SparkConf()
conf.set("spark.app.name", "natasha pritykovskaya Spark RDD app") 

sc = SparkContext(conf=conf)

In [122]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2020-09-05 20:38 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2020-09-05 20:38 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2020-09-05 20:38 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2020-09-05 20:38 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2020-09-05 20:38 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2020-09-05 20:38 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2020-09-05 20:38 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2020-09-05 20:38 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2020-09-05 20:38 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2020-09-05 20:38 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2020-09-05 20:38 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2020-09-05 20:38 /labs/laba01/ml-100k/u2.base


In [123]:
rdd = sc.textFile("/labs/laba01/ml-100k/u.data")

In [124]:
rdd.count()

100000

In [125]:
rdd.getNumPartitions()

2

In [127]:
rdd.take(5)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596']

film id for calculations 96

В поле “hist_film” нужно указать для заданного id фильма количество поставленных оценок в следующем порядке: "1", "2", "3", "4", "5". То есть сколько было единичек, двоек, троек и т.д.

In [128]:
result = {
   "hist_film": None,
   "hist_all": None
}

In [129]:
ID = '96'
result["hist_film"] = sorted(rdd.map(lambda x: x.split("\t"))
         .filter(lambda x: len(x) ==4 )
         .filter(lambda x: x[1] == ID)
         .map(lambda x: (x[2], 1))\
         .groupByKey()
         .map(lambda x: (x[0], len(x[1])))
         .collect())
result["hist_film"] = [i[1] for i in result["hist_film"]]
result["hist_film"]

[6, 20, 43, 123, 103]

In [131]:
result["hist_all"] = sorted(rdd.map(lambda x: x.split("\t"))
         .filter(lambda x: len(x) ==4 )
         .map(lambda x: (x[2], 1))\
         .groupByKey()
         .map(lambda x: (x[0], len(x[1])))
         .collect())
result["hist_all"] = [i[1] for i in result["hist_all"]]

In [132]:
result

{'hist_film': [6, 20, 43, 123, 103],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [133]:
import json

with open('../lab01.json', 'w') as fp:
    json.dump(result, fp)

In [134]:
sc.stop()